In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Setting up the wandb

In [2]:
cjvkbo

NameError: name 'cjvkbo' is not defined

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

In [6]:
import wandb
wandb.login(key=secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m020 (snehalma23m020-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Loading the inaturalist dataset 
# train set is divided into train and validation in the ratio 80-20.

In [7]:
train = '/kaggle/input/data-2/inaturalist_12K/train'

test = '/kaggle/input/data-2/inaturalist_12K/val'

# Class names

In [8]:
classes = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

# importing important libraries

In [9]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms,models
import torch.optim as optim
from collections import defaultdict
from torch.utils.data import DataLoader, Subset
import wandb
import random

# Step 1. Setup 

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = '/kaggle/input/data-2/inaturalist_12K/train'
IMAGE_SIZE = (224, 224)

# Step 2 : Setting up the seed : so that same thing is selected in every run

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)

# Step 2. Splitting the data  


In [ ]:
def stratified_split(dataset, val_fraction=0.2, seed=42): 
    random.seed(seed)
    label_to_indices = defaultdict(list)
    
   
    for idx, (_, label) in enumerate(dataset.samples):    # mapping class label to index.
        label_to_indices[label].append(idx)
    
    train_indices, val_indices = [], []
    
    for label, indices in label_to_indices.items():
        n_val = int(len(indices) * val_fraction) #the number of validation samples for that class
        random.shuffle(indices)
        val_indices.extend(indices[:n_val])
        train_indices.extend(indices[n_val:])
    
    return Subset(dataset, train_indices), Subset(dataset, val_indices)

In [ ]:

# --- 3. Dataset Loader (Stratified Split + Augmentation Toggle) ---
def prepare_data(data_dir, val_fraction=0.2, use_augmentation=True):
    base_transforms = [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
    augment_transforms = [
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10)
    ]
    transform = transforms.Compose(augment_transforms + base_transforms) if use_augmentation else transforms.Compose(base_transforms)
    
    full_dataset = datasets.ImageFolder(data_dir, transform=transform)
    return stratified_split(full_dataset, val_fraction=val_fraction, seed=42)

# --- 4. Model Preparation (ResNet50 Only) ---
def initialize_model(num_classes=10, freeze_ratio=1.0):
    model = models.resnet50(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    
    # Freeze a percentage of parameters based on freeze_ratio.
    params = list(model.parameters())
    freeze_limit = int(len(params) * freeze_ratio)
    for i, param in enumerate(params):
        param.requires_grad = (i >= freeze_limit)
    
    # Explicitly ensure classifier layers are trainable.
    for param in model.fc.parameters():
        param.requires_grad = True

    return model.to(device)

# --- 5. Training Function ---
def train_model(model, dataloaders, config):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        list(filter(lambda p: p.requires_grad, model.parameters())),
        lr=config.lr, weight_decay=config.l2_reg
    )

    for epoch in range(config.epochs):
        model.train()
        total, correct = 0, 0
        for xb, yb in dataloaders['train']:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            outputs = model(xb)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            total += yb.size(0)
            correct += (outputs.argmax(1) == yb).sum().item()
        train_acc = correct / total

        model.eval()
        total, correct = 0, 0
        with torch.no_grad():
            for xb, yb in dataloaders['val']:
                xb, yb = xb.to(device), yb.to(device)
                outputs = model(xb)
                total += yb.size(0)
                correct += (outputs.argmax(1) == yb).sum().item()
        val_acc = correct / total

        wandb.log({
            "epoch": epoch,
            "train_accuracy": train_acc * 100,
            "val_accuracy": val_acc * 100
        })
        print(f"Epoch [{epoch+1}/{config.epochs}] | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

# --- 6. WandB Sweep Config ---
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'lr': {'values': [1e-3, 1e-4,1e-2]},
        'freeze_ratio': {'values': [0.2,0.4,0.6,0.8,0.9, 1.0]},
        'batch_size': {'values': [32, 64]},
        'epochs': {'values': [5, 10,15]},
        'l2_reg': {'values': [0, 0.0005, 0.05]},
        # Only resnet50 is supported now.
        'model_name': {'values': ['resnet50']},
        'data_aug': {'values': ['yes', 'no']}
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project="fine-tune-inaturalist") 

# --- 7. Sweep Runner ---
def run_sweep():
    with wandb.init() as run:
        config = wandb.config
        set_seed()

        use_aug = config.data_aug == 'yes'
        train_data, val_data = prepare_data(DATA_PATH, val_fraction=0.2, use_augmentation=use_aug)
        dataloaders = {
            'train': DataLoader(train_data, batch_size=config.batch_size, shuffle=True),
            'val': DataLoader(val_data, batch_size=config.batch_size)
        }

        model = initialize_model(num_classes=10, freeze_ratio=config.freeze_ratio)
        train_model(model, dataloaders, config)

# --- 8. Start the Sweep Runner ---
if __name__ == '__main__':
    #sweep_id='zsg2wxr3'
    wandb.agent(sweep_id, function=run_sweep, count=30)


Create sweep with ID: 2fnqi433
Sweep URL: https://wandb.ai/snehalma23m020-iit-madras/fine-tune-inaturalist/sweeps/2fnqi433


wandb: Agent Starting Run: w0blb0vy with config:
wandb: 	batch_size: 64
wandb: 	data_aug: no
wandb: 	epochs: 15
wandb: 	freeze_ratio: 1
wandb: 	l2_reg: 0
wandb: 	lr: 0.0001
wandb: 	model_name: resnet50
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15] | Train Acc: 0.4465 | Val Acc: 0.6088
Epoch [2/15] | Train Acc: 0.6639 | Val Acc: 0.6748
Epoch [3/15] | Train Acc: 0.7081 | Val Acc: 0.6913
Epoch [4/15] | Train Acc: 0.7179 | Val Acc: 0.7024
Epoch [5/15] | Train Acc: 0.7251 | Val Acc: 0.7094
Epoch [6/15] | Train Acc: 0.7335 | Val Acc: 0.7184
Epoch [7/15] | Train Acc: 0.7404 | Val Acc: 0.7189
Epoch [8/15] | Train Acc: 0.7436 | Val Acc: 0.7279
Epoch [9/15] | Train Acc: 0.7488 | Val Acc: 0.7354
Epoch [10/15] | Train Acc: 0.7561 | Val Acc: 0.7344
Epoch [11/15] | Train Acc: 0.7579 | Val Acc: 0.7364
Epoch [12/15] | Train Acc: 0.7644 | Val Acc: 0.7409
Epoch [13/15] | Train Acc: 0.7654 | Val Acc: 0.7414
Epoch [14/15] | Train Acc: 0.7665 | Val Acc: 0.7379
Epoch [15/15] | Train Acc: 0.7690 | Val Acc: 0.7484


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▆▇▇▇▇▇▇███████
val_accuracy,▁▄▅▆▆▆▇▇▇▇▇██▇█
epoch,14
train_accuracy,76.9
val_accuracy,74.83742


...

wandb: Agent Starting Run: n2jl7bp1 with config:
wandb: 	batch_size: 64
wandb: 	data_aug: no
wandb: 	epochs: 15
wandb: 	freeze_ratio: 1
wandb: 	l2_reg: 0.0005
wandb: 	lr: 0.0001
wandb: 	model_name: resnet50


Epoch [1/15] | Train Acc: 0.4465 | Val Acc: 0.6088
Epoch [2/15] | Train Acc: 0.6639 | Val Acc: 0.6748
Epoch [3/15] | Train Acc: 0.7081 | Val Acc: 0.6913
Epoch [4/15] | Train Acc: 0.7179 | Val Acc: 0.7024
Epoch [5/15] | Train Acc: 0.7251 | Val Acc: 0.7094
Epoch [6/15] | Train Acc: 0.7335 | Val Acc: 0.7184
Epoch [7/15] | Train Acc: 0.7401 | Val Acc: 0.7189
Epoch [8/15] | Train Acc: 0.7436 | Val Acc: 0.7274
Epoch [9/15] | Train Acc: 0.7486 | Val Acc: 0.7354
Epoch [10/15] | Train Acc: 0.7561 | Val Acc: 0.7344
Epoch [11/15] | Train Acc: 0.7579 | Val Acc: 0.7364
Epoch [12/15] | Train Acc: 0.7641 | Val Acc: 0.7409
Epoch [13/15] | Train Acc: 0.7655 | Val Acc: 0.7414
Epoch [14/15] | Train Acc: 0.7666 | Val Acc: 0.7379
Epoch [15/15] | Train Acc: 0.7690 | Val Acc: 0.7479


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▆▇▇▇▇▇▇███████
val_accuracy,▁▄▅▆▆▇▇▇▇▇▇██▇█
epoch,14
train_accuracy,76.9
val_accuracy,74.78739


wandb: Agent Starting Run: cdxkeu6o with config:
wandb: 	batch_size: 64
wandb: 	data_aug: no
wandb: 	epochs: 15
wandb: 	freeze_ratio: 1
wandb: 	l2_reg: 0.05
wandb: 	lr: 0.01
wandb: 	model_name: resnet50


Epoch [1/15] | Train Acc: 0.5955 | Val Acc: 0.6683
Epoch [2/15] | Train Acc: 0.6576 | Val Acc: 0.6508
Epoch [3/15] | Train Acc: 0.6480 | Val Acc: 0.6623
Epoch [4/15] | Train Acc: 0.6452 | Val Acc: 0.6523
Epoch [5/15] | Train Acc: 0.6494 | Val Acc: 0.7029
Epoch [6/15] | Train Acc: 0.6484 | Val Acc: 0.6798
Epoch [7/15] | Train Acc: 0.6429 | Val Acc: 0.6803
Epoch [8/15] | Train Acc: 0.6502 | Val Acc: 0.6658
Epoch [9/15] | Train Acc: 0.6524 | Val Acc: 0.6488
Epoch [10/15] | Train Acc: 0.6411 | Val Acc: 0.5828
Epoch [11/15] | Train Acc: 0.6495 | Val Acc: 0.6198
Epoch [12/15] | Train Acc: 0.6481 | Val Acc: 0.6513
Epoch [13/15] | Train Acc: 0.6374 | Val Acc: 0.6438
